# Modelos de valuación de activos (CAPM y Multifactor) — Notebook de clase

**Objetivo:** estimar y comparar CAPM vs un modelo multifactor (MKT, SMB, HML, MOM), interpretar betas/alphas y decidir si hay evidencia de “alpha real” o solo riesgo disfrazado.

**Dataset:** `dataset_asset_pricing_mcf.csv` (incluye RF, mercado, factores y 5 activos).

In [ ]:
# === Setup ===
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

DATA_PATH = "dataset_asset_pricing_mcf.csv"  # si no está en el mismo folder, ajusta ruta

In [ ]:
# === Carga de datos ===
df = pd.read_csv(DATA_PATH, parse_dates=["date"]).sort_values("date").reset_index(drop=True)

# Vista rápida
display(df.head())
print("Filas/Columnas:", df.shape)
print("Rango fechas:", df["date"].min().date(), "→", df["date"].max().date())

## 1) Warm-up: retornos exceso y exploración rápida

**Idea:** En CAPM y modelos de factores normalmente trabajamos con *retornos en exceso*:
- `ASSET_EXCESS = ASSET - RF`
- `MKT_EXCESS = MKT - RF`

**Tarea:** crea las columnas de retornos en exceso para:
- Mercado (`MKT_EXCESS`)
- Un activo que tú elijas (por ejemplo `ASSET_A`)

Luego calcula:
- media y volatilidad anualizada (aprox. 252 días hábiles)

In [ ]:
# === TODO: crea retornos exceso ===
# Pista: df["MKT_EXCESS"] = df["MKT"] - df["RF"]
# Elige un activo (cambia el nombre si quieres)
ASSET_COL = "ASSET_A"

# TODO 1: define MKT_EXCESS
# df["MKT_EXCESS"] = ...

# TODO 2: define ASSET_EXCESS
# df["ASSET_EXCESS"] = ...

# === TODO: métricas anualizadas ===
PER_YEAR = 252
def annualize_mean(r, periods_per_year=PER_YEAR):
    return r.mean() * periods_per_year

def annualize_vol(r, periods_per_year=PER_YEAR):
    return r.std() * np.sqrt(periods_per_year)

# TODO 3: imprime media y vol anualizada para MKT_EXCESS y ASSET_EXCESS
# print("E[MKT_EXCESS] anual:", ...)
# print("Vol[MKT_EXCESS] anual:", ...)
# print("E[ASSET_EXCESS] anual:", ...)
# print("Vol[ASSET_EXCESS] anual:", ...)

## 2) Mini-Markowitz (opcional, rápido)

No vamos a optimizar con restricciones formales aquí, pero sí ver la intuición:

**Tarea:** construye 5,000 portafolios aleatorios con los 5 activos (`ASSET_A` a `ASSET_E`), calcula retorno y volatilidad anualizada y grafica la nube.

> Si no te da tiempo: salta a CAPM. No te juzgo (mucho).

In [ ]:
# === Portafolios aleatorios (opcional) ===
asset_cols = ["ASSET_A","ASSET_B","ASSET_C","ASSET_D","ASSET_E"]
rets = df[asset_cols]

# TODO 1: genera pesos aleatorios que sumen 1 (dirichlet)
# W = ...

# TODO 2: calcula retorno esperado anual y vol anual de cada portafolio
# Pista:
# mu = rets.mean().values
# Sigma = rets.cov().values
# port_mu = W @ mu
# port_var = np.einsum('ij,jk,ik->i', W, Sigma, W)

# TODO 3: grafica (vol vs retorno)

## 3) CAPM: estimación e interpretación

Modelo:
\[
R_i - R_f = \alpha + \beta (R_m - R_f) + \epsilon
\]

**Tareas:**
1) Ajusta el modelo CAPM (OLS) para tu activo elegido.  
2) Reporta: alpha, beta, p-values, R².  
3) Interpreta en 4–6 líneas: ¿qué significa el beta? ¿hay evidencia de alpha (p-value)?

In [ ]:
# === CAPM ===
# Asegúrate de haber creado MKT_EXCESS y ASSET_EXCESS en la sección 1.

X = sm.add_constant(df["MKT_EXCESS"])
y = df["ASSET_EXCESS"]

capm = sm.OLS(y, X).fit()
print(capm.summary())

# TODO: extrae alpha/beta/p-values/R2 en variables y muéstralos de forma limpia
# alpha = ...
# beta = ...
# p_alpha = ...
# p_beta = ...
# r2 = ...

## 4) Multifactor (MKT, SMB, HML, MOM)

Modelo:
\[
R_i - R_f = \alpha + b_m (R_m - R_f) + b_{smb} SMB + b_{hml} HML + b_{mom} MOM + \epsilon
\]

**Tareas:**
1) Ajusta el multifactor.  
2) Compara contra CAPM: R², alpha y significancia.  
3) Interpreta al menos 2 factores (ej. SMB positivo = sesgo a acciones pequeñas).

In [ ]:
# === Multifactor ===
factors = ["MKT_EXCESS", "SMB", "HML", "MOM"]
Xk = sm.add_constant(df[factors])
mf = sm.OLS(df["ASSET_EXCESS"], Xk).fit()
print(mf.summary())

# TODO: arma una tabla comparativa CAPM vs MF (coef y p-values)

## 5) Reporte final (mini-caso)

Entrega en el notebook (o en texto aquí):
- Tabla CAPM vs MF (coeficientes y p-values)
- CAPM R² vs MF R²
- Conclusión ejecutiva:
  - ¿Hay alpha? (basado en p-value + magnitud)
  - ¿Qué factores dominan el retorno en exceso?
  - 1 riesgo/limitación (estabilidad, sobreajuste, régimen, etc.)

In [ ]:
def summarize_model(m):
    return pd.DataFrame({"coef": m.params, "pval": m.pvalues})

capm_tbl = summarize_model(capm)
mf_tbl = summarize_model(mf)

display(pd.concat({"CAPM": capm_tbl, "MF": mf_tbl}, axis=1))
print("CAPM R²:", capm.rsquared)
print("MF   R²:", mf.rsquared)

# TODO: escribe tu conclusión aquí (en un markdown cell o como comentario)

## 6) Bonus (+10): estabilidad (rolling beta)

**Objetivo:** estimar beta en ventanas móviles (ej. 252 días) para ver si es estable o cambia por régimen.

Tarea:
- Calcula beta rolling (regresión por ventana) y grafícala.

In [ ]:
# === Bonus: rolling beta ===
window = 252

# TODO: calcula betas por ventana usando cov/var como aproximación rápida:
# beta_t = Cov(ASSET_EXCESS, MKT_EXCESS) / Var(MKT_EXCESS)
# Pista: usa rolling().cov() y rolling().var()

# roll_cov = df["ASSET_EXCESS"].rolling(window).cov(df["MKT_EXCESS"])
# roll_var = df["MKT_EXCESS"].rolling(window).var()
# roll_beta = roll_cov / roll_var

# plt.figure()
# plt.plot(df["date"], roll_beta)
# plt.title("Rolling beta (ventana 252)")
# plt.show()